# Detailed vs Condensed

In [ ]:
from dsdobjects import clear_memory
from peppercornenumerator import enumerate_pil
from subprocess import Popen, PIPE # simulations

import pandas as pd # analysis
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

In [ ]:
results = [] # A list of results => for dataframe

In [ ]:
# DEBUGGING -- Current State
#print(pil_condensed)
#print(pil_detailed)
#display(results)
clear_memory()

# Kotani Autocataliytic

In [ ]:
Kotani2017_F4_input = """
length a   = 22
length b   = 22
length o   = 22
length c1  = 11
length c2  = 11
length t1  = 6
length t2  = 6
length t3  = 10
length T2  = 2
length x   = 2
length y   = 2

length d1s = 16
length d1r = 2
length d2  = 6

S5 = o( b*( T2 a + c2 ) a( t2( y + ) ) c2*( c1*( t1* x* + ) ) ) d2 t3 @i 1e-7 M
S6 = y* t2* a*( b*( c2*( + x t1 c1 ) ) o*( + d1s T2 ) c2*( c1*( t1*( + x ) ) ) ) @i 1e-7 M

C1 = x t1 c1 c2 a @i 1e-9 M

P1  = t2* a*( c2*( c1*( t1*( x*( + ) ) ) ) )           @i 0 M
P2 = c2( b( a( t2( y( + ) ) ) ) )                     @i 0 M

I5  = c1 c2 o*( d2 t3 + ) b*( T2 a + c2 ) a t2 y        @i 0 M
I6 = c1( c2( o*( d2 t3 + ) b*( T2 a + c2 ) a( t2( y( + ) ) ) b*( c2*( + x t1 c1 ) ) o*( + d1s T2 ) ) ) t1* @i 0 M
P10 = x( t1( c1( c2( b( o*( + ) ) T2 a( + t2* ) ) ) ) ) @i 0 M

P8 = x t1 c1 c2 b( o*( + ) ) T2 a                     @i 0 M
P9 = d1s T2 o( c2*( c1*( t1* + ) ) ) d2 t3            @i 0 M

R = d1r( d1s( d2( + t3* ) ) ) @i 1e-7 M
D = d1r d1s d2                                              @i 0 M
RW = d1s( T2 o( c2*( c1*( t1* + ) ) ) d2( t3( + ) ) ) d1r*  @i 0 M
"""

In [ ]:
enum, pil_condensed = enumerate_pil(Kotani2017_F4_input, detailed = False, condensed = True, max_complex_size = 20, release_cutoff = 8, enumconc = 'nM')
clear_memory()

In [ ]:
enum, pil_detailed  = enumerate_pil(Kotani2017_F4_input, detailed = True, condensed = False, max_complex_size = 20, release_cutoff = 8, enumconc = 'nM')
clear_memory()

In [ ]:
def kotani_t_half(pilstring, conc):
    command = f'pilsimulator --nxy --p0 C1={conc} S5={conc} S6={conc} R={conc} --t0 1 --t8 1e7 --t-log 1000 --pyplot-labels C1 S5 S6 R D'
    #print(command)
    process = Popen(command.split(), stdin = PIPE, stdout = PIPE)
    simu, err = process.communicate(pilstring.encode())
    if process.returncode:
        print('something went wrong.')

    for line in simu.decode().split('\n'):
        if not line:
            continue
        if line[0] == '#':
            continue
        [t, C1, S5, S6, R, D] = line.split()[:6]
        #print(t, C1, S5, S6, R, D)
        if float(D) > 0.5 *  conc:
            return t, D

for c in np.logspace(np.log10(1e-2), np.log10(25), num = 30):
    dt, dD = kotani_t_half(pil_detailed, c)
    #print('detailed', c, float(dt), float(dD))
    ct, cD = kotani_t_half(pil_condensed, c)
    #print('condensed', c, float(ct), float(cD))
    print('ratio', c, float(dt)/float(ct), float(dD)/float(cD))
    results.append(['Kotani (2017)', c, float(dt)/float(ct), float(dD)/float(cD)])


# Zhang 3-way branch migration

In [ ]:
zhang_th78 = """
# Experiments Zhang 2009, 3-way strand displacement
length a = 16
length b1 = 6 
length b2 = 7 
length b3 = 7
length c = 5  # c
length d = 10 # d

# Chopped sequence for reporter reaction.
sup-sequence b = b1 b2 b3

# Inputs
X = b c 
S = a b( + d* c* )
R = a( b1( + b2* ) )

# Outputs
L = b( c( + d* ) )        @initial 0 M
Y = a b                   @initial 0 M
F = a b1                  @initial 0 M
W = a( b1( b2( b3 + ) ) ) @initial 0 M

# Intermediates
i1 = a b( + b c( + d* ) ) @initial 0 M

# Expected detailed reactions:
# X + S <=> i1
# i1 -> L + Y
# Target condensed reactions:
# X + S -> L + Y
# R + Y -> F + W
"""

In [ ]:
enum, pil_condensed = enumerate_pil(zhang_th78, detailed = False, condensed = True, max_complex_size = 20, release_cutoff = 8, enumconc = 'nM')
clear_memory()

In [ ]:
enum, pil_detailed  = enumerate_pil(zhang_th78, detailed = True, condensed = False, max_complex_size = 20, release_cutoff = 8, enumconc = 'nM')
clear_memory()

In [ ]:
def zhang_t_half(pilstring, conc):
    command = f'pilsimulator --nxy --p0 X={conc} S={conc} R={conc} --t0 0.1 --t8 1e7 --t-log 10000 --pyplot-labels X S R F'
    #print(command)
    process = Popen(command.split(), stdin = PIPE, stdout = PIPE)
    simu, err = process.communicate(pilstring.encode())
    if process.returncode:
        print('something went wrong.')

    for line in simu.decode().split('\n'):
        if not line:
            continue
        if line[0] == '#':
            continue
        [t, X, S, R, F] = line.split()[:5]
        #print(t, X, S, R, F)
        if float(F) > 0.5 *  conc:
            return t, F

for c in np.logspace(np.log10(1e-2), np.log10(1e5), num = 30):
    dt, dD = zhang_t_half(pil_detailed, c)
    #print('detailed', c, float(dt), float(dD))
    ct, cD = zhang_t_half(pil_condensed, c)
    #print('condensed', c, float(ct), float(cD))
    print('ratio', c, float(dt)/float(ct), float(dD)/float(cD))
    results.append(['Zhang (2009) (5,10)', c, float(dt)/float(ct), float(dD)/float(cD)])

In [ ]:
df = pd.DataFrame(results, columns=['system', 'initial-concentration', 'T50', '[Dye]'])
display(df)

In [ ]:
df['T50']
g = sns.lmplot(x="initial-concentration", y="T50", hue="system", data=df, fit_reg = False, logistic=False, truncate=False)
plt.gcf().set_size_inches(8.5,4)

#g.set_titles(row_template = '{row_name}', col_template = '{col_name}')
plt.gca().set_xlabel('Initial concentration [nM]', fontsize=10)
plt.gca().set_ylabel('$t_{50}$ detailed / $t_{50}$ condensed', fontsize=10)
#plt.gca().axhline(y=5, linewidth=1, color='black', linestyle='--')

plt.xscale('log')
plt.xlim((0.01, 1.1e4))
plt.ylim((0, 8))

#sns.lineplot(df, palette="tab10", hue = 'semantics', linewidth=2.5)
#plt.savefig('condensation.pdf', bbox_inches='tight')